In [9]:
from PIL import Image
import requests
from io import BytesIO
import pandas as pd
from __future__ import print_function
import binascii
import struct
from PIL import Image
import numpy as np
import scipy
import scipy.misc
import scipy.cluster

In [30]:
df = pd.read_csv("hallmark_card_rgb_codes_sorted.csv")
df= df.drop(['Unnamed: 0', 'index', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1'],axis=1)

In [131]:
def getHue(url):
    
    # use the url to open the image and save it
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    
    # get hex codes and pixel counts of 5 most dominant colors in image
    NUM_CLUSTERS = 5

    im = img
    im = im.resize((150, 150))      # optional, to reduce time
    ar = np.asarray(im)
    shape = ar.shape
    ar = ar.reshape(scipy.product(shape[:2]), shape[2]).astype(float)
    
    codes, dist = scipy.cluster.vq.kmeans(ar, NUM_CLUSTERS)

    vecs, dist = scipy.cluster.vq.vq(ar, codes)         # assign codes
    counts, bins = scipy.histogram(vecs, len(codes))    # count occurrences

    #index_max = scipy.argmax(counts)                    # find most frequent
    #peak = codes[index_max]
    #colour = binascii.hexlify(bytearray(int(c) for c in peak)).decode('ascii')
    #print('most frequent is %s (#%s)' % (peak, colour))

    # put the top 5 hex codes and their pixel counts in a df, sort so dominant is first
    results = pd.DataFrame({'count':counts,'hex':list(codes)})
    results = results.sort_values('count',ascending=False)
    results = results.reset_index()
    del results['index']

    # if the dominant color is white, use the second most dominant color instead
    if ((results['hex'][0][0] >= 240) & (results['hex'][0][0] >= 240) & (results['hex'][0][0] >= 240)): 
        # convert rgb values to 0-1 scale
        R = results['hex'][1][0] / 255
        G = results['hex'][1][1] / 255
        B = results['hex'][1][2] / 255
    else: # the dominant color isn't white, so use it
        # convert rgb values to 0-1 scale
        R = results['hex'][0][0] / 255
        G = results['hex'][0][1] / 255
        B = results['hex'][0][2] / 255

    # put them in a list
    rgb = [R,G,B]
    
    # calculate hue depending on rgb's max and min
    if max(rgb) == min(rgb):
        hue = 0
    elif max(rgb) == R:
        hue = (G-B)/(max(rgb)-min(rgb))
    elif max(rgb) == G:
        hue = 2 + (B-R)/(max(rgb)-min(rgb))
    else:
        hue = 4 + (R-G)/(max(rgb)-min(rgb))

    # multply by 60 to convert to degrees on the color circle
    hue = hue * 60

    # if hue is negative, add 360 because a cirle has 360 degrees
    if hue < 0:
        hue = hue + 360

    return hue

In [132]:
df['hue'] = df['s3_link'].apply(getHue)
df = df.sort_values('hue')
df.to_csv("C:\\Users\\glol7001\\Documents\\hallmark_card_sorted_by_hue.csv")